# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [76]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [77]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2


Exception ignored in: <function tqdm.__del__ at 0x0000024371D119E0>
Traceback (most recent call last):
  File "c:\Users\bouml\anaconda3\envs\env\Lib\site-packages\tqdm\std.py", line 1145, in __del__
    self.close()
  File "c:\Users\bouml\anaconda3\envs\env\Lib\site-packages\tqdm\notebook.py", line 283, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews


In [78]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

,reviews
0,Not Verified | Food was lousy. Who ever is pl...
1,✅ Trip Verified | Had the worst experience. Th...
2,✅ Trip Verified | The ground staff were not h...
3,✅ Trip Verified | Second time BA Premium Econ...
4,Not Verified | They changed our Flights from ...


In [79]:
df.to_csv("data/BA_reviews.csv")

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [80]:
df['reviews'] = df['reviews'].str.split('|' , expand=True)[1]

In [81]:
df.head()

,reviews
0,Food was lousy. Who ever is planning the Asi...
1,Had the worst experience. The flight from Lon...
2,The ground staff were not helpful. Felt like...
3,Second time BA Premium Economy in a newer ai...
4,They changed our Flights from Brussels to Lo...


# VADER method #
We'll use NLTK's SentimentIntensityAnalyzer to get the neg/neu/pos score 
* removing stop words

In [82]:
example = df["reviews"][0]
print(example)

  Food was lousy. Who ever is planning the Asian Hindu Vegetarian meal is clueless as to what this meal includes. The snack was also lousy. It took us 2 hours just to go from T5 to T3 and clear security check. The place was chaotic. We get to our boarding gate and again a long line with 40 minutes wait for someone to check passport and boarding pass. Seats in Premium economy was ok, but when the passenger in front reclines, there is very little space for the window seat passenger to cross over. Overall not a good airline.


In [83]:
nltk.word_tokenize(example)

['Food',
 'was',
 'lousy',
 '.',
 'Who',
 'ever',
 'is',
 'planning',
 'the',
 'Asian',
 'Hindu',
 'Vegetarian',
 'meal',
 'is',
 'clueless',
 'as',
 'to',
 'what',
 'this',
 'meal',
 'includes',
 '.',
 'The',
 'snack',
 'was',
 'also',
 'lousy',
 '.',
 'It',
 'took',
 'us',
 '2',
 'hours',
 'just',
 'to',
 'go',
 'from',
 'T5',
 'to',
 'T3',
 'and',
 'clear',
 'security',
 'check',
 '.',
 'The',
 'place',
 'was',
 'chaotic',
 '.',
 'We',
 'get',
 'to',
 'our',
 'boarding',
 'gate',
 'and',
 'again',
 'a',
 'long',
 'line',
 'with',
 '40',
 'minutes',
 'wait',
 'for',
 'someone',
 'to',
 'check',
 'passport',
 'and',
 'boarding',
 'pass',
 '.',
 'Seats',
 'in',
 'Premium',
 'economy',
 'was',
 'ok',
 ',',
 'but',
 'when',
 'the',
 'passenger',
 'in',
 'front',
 'reclines',
 ',',
 'there',
 'is',
 'very',
 'little',
 'space',
 'for',
 'the',
 'window',
 'seat',
 'passenger',
 'to',
 'cross',
 'over',
 '.',
 'Overall',
 'not',
 'a',
 'good',
 'airline',
 '.']

In [84]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [85]:
sia.polarity_scores(example)

{'neg': 0.11, 'neu': 0.842, 'pos': 0.049, 'compound': -0.7476}

In [86]:
df.reset_index(inplace=True)
df.head()

,index,reviews
0,0,Food was lousy. Who ever is planning the Asi...
1,1,Had the worst experience. The flight from Lon...
2,2,The ground staff were not helpful. Felt like...
3,3,Second time BA Premium Economy in a newer ai...
4,4,They changed our Flights from Brussels to Lo...


In [87]:
# df.drop(columns=["level_0"], inplace=True)
df.rename(columns={"index": "review_id"}, inplace=True)

In [88]:
# run the polarity score on all reviews
res = {}
for i, row in df.iterrows():
    text = row["reviews"]
    myid = row["review_id"]
    res[myid] = sia.polarity_scores(text)
    

In [89]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={"index": "review_id"})
vaders = vaders.merge(df, how='left' )
vaders

,review_id,neg,neu,pos,compound,reviews
0,0,0.110,0.842,0.049,-0.7476,Food was lousy. Who ever is planning the Asi...
1,1,0.110,0.866,0.023,-0.8604,Had the worst experience. The flight from Lon...
2,2,0.166,0.834,0.000,-0.8537,The ground staff were not helpful. Felt like...
3,3,0.052,0.677,0.270,0.9714,Second time BA Premium Economy in a newer ai...
4,4,0.096,0.866,0.038,-0.8055,They changed our Flights from Brussels to Lo...
...,...,...,...,...,...,...
995,995,0.074,0.722,0.204,0.9878,Delhi to London. Having read many negative r...
996,996,0.042,0.867,0.091,0.9288,When you travel British Airways its like you ...
997,997,0.000,0.656,0.344,0.9768,British Airways gets plenty of well deserved...
998,998,0.107,0.763,0.130,0.8053,BA First Class is not even that any more. St...
